In [13]:
import itertools
import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
def get_data(filename):
    ids = []
    data = []
    labels = []
    with open(filename, 'rb') as f:
        while 1:
            try:
                d = pickle.load(f)
                features = []
                for from_tok, to_tok in itertools.product(['A', 'B', 'P'], repeat=2):
                    if from_tok != to_tok:
                        lb = from_tok + to_tok
                        if len(d[lb]) == 0:
                            break
                        max_att = d[lb][0]
                        for att in d[lb][1:]:
                            max_att = np.maximum(max_att, att)
                        d[lb] = max_att
                        features.append(max_att.flatten())
                if len(d[lb]) == 0:
                    continue
                labels.append(d['label'])
                ids.append(d['ID'])
                data.append(np.concatenate(features))
            except EOFError:
                break
    return ids, data, labels

In [3]:
IDs_train1, X_train1, Y_train1 = get_data('./data/large-atts-gap-validation.pkl')
IDs_train2, X_train2, Y_train2 = get_data('./data/large-atts-gap-test.pkl')
IDs_train = IDs_train1 + IDs_train2
X_train = X_train1 + X_train2
Y_train = Y_train1 + Y_train2
IDs_test, X_test, Y_test = get_data('./data/large-atts-gap-development.pkl')

In [4]:
print(len(X_train), len(X_test))

2453 1998


In [18]:
X_train[0].shape

(2304,)

In [5]:
from sklearn.linear_model import L

In [8]:
clf = La(cv=5)
clf.fit(X_train, Y_train)

RidgeClassifierCV(alphas=array([ 0.1,  1. , 10. ]), class_weight=None, cv=5,
         fit_intercept=True, normalize=False, scoring=None,
         store_cv_values=False)

In [11]:
clf.score(X_test, Y_test)

0.8823823823823824

In [12]:
clf.predict(X_test, pr)

array(['A', 'A', 'B', ..., 'A', 'A', 'B'], dtype='<U7')

In [20]:
clf.coef_.shape

(3, 2304)

In [22]:
from tpot import TPOTClassifier

In [24]:
tpot = TPOTClassifier(generations=5, population_size=20, verbosity=2)
tpot.fit(X_train, Y_train)
# print(tpot.score(X_test, Y_test))

Generation 1 - Current best internal CV score: 0.8842360859478798
Generation 2 - Current best internal CV score: 0.8858637478706701
Generation 3 - Current best internal CV score: 0.8891107418189996
Generation 4 - Current best internal CV score: 0.8919753934835214
Generation 5 - Current best internal CV score: 0.8919753934835214

Best pipeline: ExtraTreesClassifier(input_matrix, bootstrap=True, criterion=gini, max_features=0.7500000000000001, min_samples_leaf=1, min_samples_split=18, n_estimators=100)


ValueError: could not convert string to float: 'A'

In [27]:
tpot.predict_proba(X_test)

array([[9.33526167e-01, 3.08144917e-02, 3.56593411e-02],
       [9.40833333e-01, 3.00000000e-02, 2.91666667e-02],
       [6.44522145e-03, 9.92999223e-01, 5.55555556e-04],
       ...,
       [8.19467252e-01, 1.02233003e-01, 7.82997453e-02],
       [9.67215909e-01, 3.04507576e-02, 2.33333333e-03],
       [1.96330237e-01, 2.35556706e-01, 5.68113057e-01]])

In [28]:
tpot.predict(X_test)

array(['A', 'A', 'B', ..., 'A', 'A', 'Neither'], dtype='<U7')

In [35]:
Y_test_ones = np.array([[int(x=='A'), int(x=='B'), int(x=='Neither')] for x in Y_test])

In [36]:
Y_pred = tpot.predict_proba(X_test)

In [44]:
from sklearn.metrics import log_loss
log_loss(Y_test_ones, Y_pred)

0.3565487192351781

In [54]:
submission = pd.read_csv('input/sample_submission_stage_1.csv', index_col='ID')

In [56]:
for _id, pred in zip(IDs_test, Y_pred):
    submission.loc[_id, 'A'] = pred[0]
    submission.loc[_id, 'B'] = pred[1]
    submission.loc[_id, 'NEITHER'] = pred[2]
submission.to_csv(f'output/TPOT.csv')